In [ ]:
# lr_preprocess.py

import os
import numpy as np
import nibabel as nib
import torchio as tio
import matplotlib.pyplot as plt

In [ ]:
def resample_nii(lr_path, hr_path, output_path):
    """
    Resample the LR image to match the voxel spacing of the HR image
    using TorchIO, then save the resampled image.
    """
    lr_image = tio.ScalarImage(lr_path)
    hr_image = tio.ScalarImage(hr_path)

    # Create a transform that resamples to match HR voxel spacing
    resample_transform = tio.Resample(target=hr_image)
    lr_upsampled = resample_transform(lr_image)

    lr_upsampled.save(output_path)
    print(f"Upsampled LR saved to: {output_path}")

def load_nifti(filepath):
    """
    Load a NIfTI file and return the 3D NumPy array plus its affine.
    """
    nii = nib.load(filepath)
    data = nii.get_fdata()  # shape could be [D, H, W], depending on orientation
    affine = nii.affine
    return data, affine

def save_nifti(data, affine, out_path):
    """
    Save a NumPy array + affine as a NIfTI file.
    """
    nii_img = nib.Nifti1Image(data, affine)
    nib.save(nii_img, out_path)
    print(f"Saved NIfTI to: {out_path}")


def extract_axial_slices(coronal_data):
    """
    If the first dimension is 'depth' in the coronal orientation,
    we treat each index of the first dimension as an 'axial' slice.
    """
    axial_slices = []
    for i in range(coronal_data.shape[2]):
        slice_2d = coronal_data[:, :, i]  # shape [H, W]
        axial_slices.append(slice_2d)
    return axial_slices

def extract_axial_slices_from_axial(axial_data):
    """
    Given an axial volume (shape [D, H, W]), extract axial slices by iterating over the first dimension.
    Each slice is volume[i, :, :] resulting in shape [H, W].
    """

    axial_slices = []
    for i in range(axial_data.shape[2]):
        slice_2d = axial_data[:, :, i]  # shape [H, W]
        axial_slices.append(slice_2d)
    return axial_slices


def extract_coronal_slices(axial_data):
    """
    Given a 3D volume that is oriented 'axially' (dimension 0 = axial),
    extract coronal slices by slicing along dimension 1.
    
    axial_data: NumPy array with shape [D, H, W]
      - D (dim 0) = number of axial slices (superior-inferior)
      - H (dim 1) = coronal dimension (anterior-posterior)
      - W (dim 2) = sagittal dimension (left-right)

    Returns a list of 2D slices (each shaped [D, W]).
    """
    coronal_slices = []
    coronal_dim = axial_data.shape[1]  # dimension 1 = coronal direction
    for i in range(coronal_dim):
        # Each slice has shape [D, W] => (axial x sagittal)
        slice_2d = axial_data[:, i, :] #axial_data[:, i, :]
        coronal_slices.append(slice_2d)
    return coronal_slices

def extract_coronal_slices_from_coronal(coronal_data):
    """
    Given a coronal volume (shape [D, H, W]), extract coronal slices by iterating over the second dimension.
    Each slice is volume[:, i, :] resulting in shape [D, W].
    """
    coronal_slices = []
    coronal_dim = coronal_data.shape[1]  # dimension 1 = coronal direction
    for i in range(coronal_dim):
        # Each slice has shape [D, W] => (axial x sagittal)
        slice_2d = coronal_data[:, i, :] #axial_data[:, i, :]
        coronal_slices.append(slice_2d)
    return coronal_slices

def visualize_slice(slice_2d, title="Slice"):
    """
    Display a 2D NumPy array as a grayscale image using Matplotlib.
    """
    plt.figure(figsize=(5,5))
    # If it looks flipped, you can transpose or set origin='lower'
    plt.imshow(slice_2d, cmap="gray")
    plt.title(title)
    plt.axis("off")
    plt.show()


def main():

    # --- File paths ---
    # Low-resolution volumes
    lr_coronal_path = "../data/sub-OAS30001_ses-d0129_run-01_T1w_coronal_LR.nii"
    lr_axial_path   = "../data/sub-OAS30001_ses-d0129_run-01_T1w_axial_LR.nii"

    coronal_data, coronal_affine = load_nifti(lr_coronal_path)
    axial_data, axial_affine   = load_nifti(lr_axial_path)
    
    print("original coronal volume shape:", coronal_data.shape)
    print("original axial volume shape:", axial_data.shape)
    
    # High-resolution volume (target)
    hr_path = "../data/sub-OAS30001_ses-d0129_run-01_T1w.nii"
    
    # target high solution shape check
    target_data, target_affine = load_nifti(hr_path)
    print()
    print("Target 3D shape: ", target_data.shape)

    # Output paths for upsampled LR volumes
    lr_coronal_upsampled_path = "../data/sub-OAS30001_ses-d0129_run-01_T1w_coronal_LR_upsampled.nii"
    lr_axial_upsampled_path   = "../data/sub-OAS30001_ses-d0129_run-01_T1w_axial_LR_upsampled.nii"

    # 1) Resample LR volumes to match HR spacing
    resample_nii(lr_coronal_path, hr_path, lr_coronal_upsampled_path)
    resample_nii(lr_axial_path, hr_path, lr_axial_upsampled_path)

    # 2) Load the upsampled LR volumes
    coronal_data, coronal_affine = load_nifti(lr_coronal_upsampled_path)
    axial_data, axial_affine   = load_nifti(lr_axial_upsampled_path)
    print()
    
    print("Upsampled coronal volume shape:", coronal_data.shape)
    print("Upsampled axial volume shape:", axial_data.shape)
    print()
    # ------------------
    # EXTRACT FROM CORONAL (Upsampled):
    # ------------------
    # 3) From the coronal volume, extract axial and sagittal slices
    axial_slices_from_coronal   = extract_axial_slices(coronal_data)
    sagittal_slices_from_coronal = extract_sagittal_slices(coronal_data)

    print(f"Extracted {len(axial_slices_from_coronal)} axial slices from coronal volume")
    print(f"Extracted {len(sagittal_slices_from_coronal)} sagittal slices from coronal volume")
    print()
    # 4) Visualize a few slices
    if len(axial_slices_from_coronal) > 50:
        visualize_slice(axial_slices_from_coronal[128], title="Axial Slice #128 (from Coronal)")

    if len(sagittal_slices_from_coronal) > 50:
        visualize_slice(sagittal_slices_from_coronal[128], title="Sagittal Slice #128 (from Coronal)")

    
    # ------------------
    # EXTRACT FROM AXIAL (Upsampled):
    # ------------------
    # 5) From the axial volume, extract coronal slices
    coronal_slices_from_axial = extract_coronal_slices(axial_data)
    print(f"Extracted {len(coronal_slices_from_axial)} coronal slices from axial volume")

    # 6) Visualize a middle slice
    mid_idx = len(coronal_slices_from_axial) // 2
    visualize_slice(coronal_slices_from_axial[128], title=f"Coronal Slice #{mid_idx} (from Axial)")

    np.save("../data/lr_slices_axial.npy", axial_slices_from_coronal)
    print("Saved data/lr_slices_axial.npy")

    np.save("../data/lr_slices_sagittal.npy", sagittal_slices_from_coronal)
    print("Saved data/lr_slices_sagittal.npy")

    np.save("../data/lr_slices_coronal.npy",  coronal_slices_from_axial)
    print("Saved data/lr_slices_coronal.npy")



    hr_axial_slices = extract_axial_slices(target_data)
    hr_sagittal_slices = extract_sagittal_slices(target_data)
    hr_coronal_slices = extract_coronal_slices(target_data)

    mid_idx = len(hr_axial_slices) // 2
    visualize_slice(hr_axial_slices[128], title=f"Target")
    mid_idx = len(hr_sagittal_slices) // 2
    visualize_slice(hr_sagittal_slices[128], title=f"Target")
    mid_idx = len(hr_coronal_slices) // 2
    visualize_slice(hr_coronal_slices[128], title=f"Target")

    np.save("../data/hr_slices_axial.npy", hr_axial_slices)
    print("Saved data/hr_slices_axial.npy")
    
    np.save("../data/hr_slices_sagittal.npy", hr_sagittal_slices)
    print("Saved data/hr_slices_sagittal.npy")
    
    np.save("../data/hr_slices_coronal.npy", hr_coronal_slices)
    print("Saved data/hr_slices_coronal.npy")


if __name__ == "__main__":
    main()
